# Zadanie 1.

Napisz program wyznaczający iteracyjnie epsilony maszynowe, dla wszystkich dostępnych typów zmiennopozycyjnych: Float16, Float32, Float64, zgodnych ze standardem IEEE 754.

In [25]:
function macheps(my_type)
    
    macheps::my_type = 1.0
    one::my_type = 1.0
    divider::my_type = 2.0
    
    while one + macheps > one && one + macheps == 1 + macheps
        macheps /= divider
    end
    
    return macheps * 2
end

macheps (generic function with 1 method)

In [26]:
println(macheps(Float16), " ", eps(Float16))
println(macheps(Float32), " ", eps(Float32))
println(macheps(Float64), " ", eps(Float64))

0.000977 0.000977
1.1920929e-7 1.1920929e-7
2.220446049250313e-16 2.220446049250313e-16


<br/><br/>

![Porównanie machepsa z informacjami zawartymi we float.h](img/e.png)

<center><i>Informacje o machepsie zawarte w bibliotece float.h</i></center>

<br/><br/>

Wyniki zwracane przez moją funkcję *macheps* są identyczne z wartościami zwracanymi przez funkcję Julii - *eps*
Uzyskane epsilony maszynowe są rzeczywistą odległością liczby 1.0 od kolejnej liczby x (x > 1) w podanych typach zmiennopozycyjnych. 

Czy liczba macheps ma związek z precyzją arytmetyki?

In [2]:
println("Precyzja Float16: ", 2^-11)

Precyzja Float16: 0.00048828125


In [3]:
println("Precyzja Float32: ", 2^-24)

Precyzja Float32: 5.960464477539063e-8


In [5]:
println("Precyzja Float64: ", 2^-53)

Precyzja Float64: 1.1102230246251565e-16


Precyzja arytmetyki mówi o tym ile cyfr znaczących (stanowiących mantysę w IEEE 754) jest reprezentowanych dokładnie, innymi słowy, na którym mniejscu znajduje się ostatni bit reprezentowany dokładnie. Precyzja jest związana z długością mantysy (która wynosi kolejno 10, 23, 52 bity/ów dla Float16, Float32, Float64).
Z pomocą powyższych obliczeń można zauważyć, że epsilon maszynowy różni się o jeden rząd wielkości (jest 2x większy) od precyzji poszczególnych typów, zatem mieści się w mantysie.

Następnym zadaniem jest wyznaczenie iteracyjnie liczby maszynowej *eta*, czyli najmniejszej możliwej liczby, 
dla poszczególnych typów zmiennoprzecinkowych.

In [27]:
function eta(my_type)
    
    eta::my_type = 1.0
    divider::my_type = 2.0
    zero::my_type = 0.0
    
    while eta / divider > zero
        eta /= divider
    end
    
    return eta
end

eta (generic function with 1 method)

In [28]:
println(eta(Float16), " ", nextfloat(Float16(0.0)))
println(eta(Float32), " ", nextfloat(Float32(0.0)))
println(eta(Float64), " ", nextfloat(Float64(0.0)))

6.0e-8 6.0e-8
1.0e-45 1.0e-45
5.0e-324 5.0e-324


Otrzymane przeze mnie wyniki są identyczne z wartościami zwracanymi przez *nextfloat*. 
Ponadto *eta* jest równa $MIN_{SUB}$.

In [42]:
println(floatmin(Float32))
println(floatmin(Float64))

1.1754944e-38
2.2250738585072014e-308


Funkcja *floatmin* zwraca minimum znormalizowane dla odpowiedniej arytmetyki. 

Następnie wyznaczę iteracyjnie maksymalną liczbę dla wszystkich typów zmiennopozycyjnych.

In [30]:
function max_func(my_type)
    
    my_max::my_type = 0.0
    helper::my_type = 1.0
    two::my_type = 2.0
    zero::my_type = 0.0
    
    while my_max + helper < two
        my_max += helper
        helper /= two
    end
    
    while !isinf(my_max * two)
        my_max *= two
    end
    
    return my_max
end

max_func (generic function with 1 method)

In [31]:
println(max_func(Float16), " ", floatmax(Float16))
println(max_func(Float32), " ", floatmax(Float32))
println(max_func(Float64), " ", floatmax(Float64))

6.55e4 6.55e4
3.4028235e38 3.4028235e38
1.7976931348623157e308 1.7976931348623157e308


Maksymalna liczba wyliczona przeze mnie pokrywa się z wartościami przedstawionymi na wykładzie. 

![Porównanie machepsa z informacjami zawartymi we float.h](img/max.png)

<center><i>Podstawowe informacje o każdym formacie, przedstawione na wykładzie</i></center>


Podsumowując, należy pamiętać, że każdy typ zmiennopozycyjny arytmetyki IEEE 754 ma skończoną dokładność, co bezpośrednio wpływa na otrzymywane wyniki.

# Zadanie 2 

Sprawdź czy wzór na epsilon maszynowy, podany przez Kahana jest prawdziwy.

In [83]:
function kahan(my_type)
    
    three::my_type = 3
    one::my_type = 1
    four::my_type = 4
    
    return three * (four/three - one) - one
end

kahan (generic function with 1 method)

In [85]:
println(kahan(Float16), " ", eps(Float16))
println(kahan(Float32), " ", eps(Float32))
println(kahan(Float64), " ", eps(Float64))

-0.000977 0.000977
1.1920929e-7 1.1920929e-7
-2.220446049250313e-16 2.220446049250313e-16


Wzór Kahana rzeczywiście zwraca nam epsilony maszynowe dla poszczególnych typów zmiennoprzecinkowych. Zważywszy jednak na definicję machepsa (który jest najmniejszą odległością 1.0 od x (x > 1)), należy na wzór nałożyć wartość bezwzględną, żeby wynik był formalnie poprawny.

# Zadanie 3

Czy liczby zmiennopozycyjne są równomiernie rozmieszczone w przedziale $[1,2]$ z krokiem $\delta = 2^{-52}$?

Jak rozmieszczone są w przedziałach $[\frac{1}{2}, 1]$ i $[2,4]$?

In [79]:
function numbers(start, fin, delta, results=5)
    
    for i = floor(start):fin - 1 
        println("Starting point: ", start)
        k::Float64 = 1.0
        while k <= Float64(results)
            println(bitstring(Float64(start) + k*Float64(delta)))
            k += Float64(1.0)
        end
        start += 1
    end
end


numbers (generic function with 5 methods)

In [80]:
numbers(1, 2, 2^-52)

Starting point: 1
0011111111110000000000000000000000000000000000000000000000000001
0011111111110000000000000000000000000000000000000000000000000010
0011111111110000000000000000000000000000000000000000000000000011
0011111111110000000000000000000000000000000000000000000000000100
0011111111110000000000000000000000000000000000000000000000000101


In [81]:
numbers(0.5,1, 2^-53)

Starting point: 0.5
0011111111100000000000000000000000000000000000000000000000000001
0011111111100000000000000000000000000000000000000000000000000010
0011111111100000000000000000000000000000000000000000000000000011
0011111111100000000000000000000000000000000000000000000000000100
0011111111100000000000000000000000000000000000000000000000000101


In [82]:
numbers(2, 4, 2^-51)

Starting point: 2
0100000000000000000000000000000000000000000000000000000000000001
0100000000000000000000000000000000000000000000000000000000000010
0100000000000000000000000000000000000000000000000000000000000011
0100000000000000000000000000000000000000000000000000000000000100
0100000000000000000000000000000000000000000000000000000000000101
Starting point: 3
0100000000001000000000000000000000000000000000000000000000000001
0100000000001000000000000000000000000000000000000000000000000010
0100000000001000000000000000000000000000000000000000000000000011
0100000000001000000000000000000000000000000000000000000000000100
0100000000001000000000000000000000000000000000000000000000000101


Na podstawie powyższych eksperymentów można stwierdzić, że liczby w podanych przedziałach są równomiernie rozłożone (każda kolejna liczba jest o 1 większa), zgodnie ze wzorem $x = 1 + k\delta$, dla różnych kroków $\delta$. Krok dla danego przedziału zależy od cechy - rośnie wraz z jej wzrostem. Stały odstęp między liczbami z podanych przedziałów, wynika z niezmiennej liczby bitów mantysy. Można zatem stwierdzić, że w każdym takim przedziale można reprezentować tyle samo liczb.
Dla przedziału $[1,2]$, równomierne rozmieszczenie dla kroku $\delta = 2^{-52}$, 
dla $[\frac{1}{2}, 1]$, równomierne rozmieszczenie dla kroku $\delta = 2^{-53}$, 
dla przedziału $[2,4]$, równomierne rozmieszczenie dla kroku $\delta = 2^{-51}$.

# Zadanie 4

Znajdź najmniejszą liczbę w przedziale $1 < x < 2$, która nie jest odwracalna, tzn. $x * \frac{1}{x} \ne 1 $

In [2]:
function min_reversible(start::Float64)
    result::Float64 = start
    one::Float64 = 1.0
    
    while result * (one/result) == one && result < 2.0
        result = nextfloat(result)  
    end
    
    return result
end

min_reversible (generic function with 1 method)

In [11]:
x = min_reversible(1.0)
println("Wynik: ", x, ", ponieważ x * 1/x = ", x * (Float64(1.0)/x))

0.9999999427710061
Wynik: 1.000000057228997, ponieważ x * 1/x = 0.9999999999999999


In [4]:
x = min_reversible(nextfloat(0.0))
println("Wynik: ", x, ", ponieważ x * 1/x = ", x * (Float64(1.0)/x))

Wynik: 5.0e-324, ponieważ x * 1/x = Inf


Na podstawie powyższych obliczeń można zauważyć, że w arytmetyce Float64, nierówność  $x * \frac{1}{x} \ne 1 $ jest prawdziwa dla niektórych liczb. Zjawisko to występuje, ponieważ następuje redukcja cyfr znaczących, wynikająca z ograniczeń precyzji zadanej arytmetyki. Problem ten będzie najbardziej widoczny na liczbach rzeczywistych, posiadających dziesiętne rozwinięcie skończone, a binarne nieskończone. 

# Zadanie 5

Napisz program do obliczania iloczynu skalarnego dwóch wektorów, wykorzystując 4 różne algorytmy sumowania.

In [6]:
"Dodawanie w przód"
function front(vector_1, vector_2, my_type)
  sum::my_type = 0.0
  for i = 1:length(vector_1)
    sum += my_type(vector_1[i]) * my_type(vector_2[i])
  end
  return sum
end

"Dodawanie w tył"
function back(vector_1, vector_2, my_type)
  sum::my_type = 0.0
  for i = length(vector_1):-1:1
    sum += my_type(vector_1[i]) * my_type(vector_2[i])
  end
  return sum
end

"Dodawanie od największego do najmniejszego"
function descending(vector_1, vector_2, my_type)
    
    positives = zeros(my_type, 0)
    negatives = zeros(my_type, 0)
    
    for i = 1 : length(vector_1)
        
        mul = my_type(vector_1[i]) * my_type(vector_2[i])
        
        if mul < 0.0
            push!(negatives, mul)
        else
            push!(positives, mul)
        end
    end
    
    return my_type(sum(sort(positives, rev=true)) + sum(sort(negatives, rev=true)))
    
    
end

"Dodawanie od najmniejszego do największego"
function ascending(vector_1, vector_2, my_type)
    
    positives = zeros(my_type, 0)
    negatives = zeros(my_type, 0)
    
    for i = 1 : length(vector_1)
        
        mul = my_type(vector_1[i]) * my_type(vector_2[i])
        
        if mul < 0.0
            push!(negatives, mul)
        else
            push!(positives, mul)
        end
    end
    
    return my_type(sum(sort(positives)) + sum(sort(negatives)))
        
end

ascending

In [7]:
x =  [2.718281828,-3.141592654,1.414213562,0.5772156649,0.3010299957]
y =  [1486.2497,878366.9879,-22.37492,4773714.647,0.000185049]
result = -1.00657107000000 * (1/10)^11

println("Float32")
f = front(x, y, Float32)
b = back(x, y, Float32)
d = descending(x, y, Float32)
a = ascending(x, y, Float32)

println("W przód: ", f, ", błąd: ", abs(result - f))
println("W tył: ", b, ", błąd: ", abs(result - b))
println("Malejąco: ", d, ", błąd: ", abs(result - d))
println("Rosnąco: ", a, ", błąd: ", abs(result - a))


println("\nFloat64")
f = front(x, y, Float64)
b = back(x, y, Float64)
d = descending(x, y, Float64)
a = ascending(x, y, Float64)

println("W przód: ", f, ", błąd: ", abs(result - f))
println("W tył: ", b, ", błąd: ", abs(result - b))
println("Malejąco: ", d, ", błąd: ", abs(result - d))
println("Rosnąco: ", a, ", błąd: ", abs(result - a))


Float32
W przód: -0.4999443, błąd: 0.49994429944939167
W tył: -0.4543457, błąd: 0.4543457031149343
Malejąco: -0.5, błąd: 0.4999999999899343
Rosnąco: -0.5, błąd: 0.4999999999899343

Float64
W przód: 1.0251881368296672e-10, błąd: 1.1258452438296672e-10
W tył: -1.5643308870494366e-10, błąd: 1.4636737800494365e-10
Malejąco: 0.0, błąd: 1.0065710700000004e-11
Rosnąco: 0.0, błąd: 1.0065710700000004e-11


Na podstawie powyższego eksperymentu, łatwo zauważyć, że zarówno precyzja (typ zmiennoprzecinkowy), jak i wykorzystywany algorytm sumowania mają wpływ na otrzymywane wyniki. Należy zatem pamiętać, że dodawanie w typach zmiennoprzecinkowych nie jest przemienne!

# Zadanie 6

Policz w arytmetyce Float64 wartości następujących funkcji: $f(x) = \sqrt{x^2 + 1} - 1$,  $g(x) = \frac{x^2}{\sqrt{x^2 + 1} + 1}$ dla $x = 8^{-n}$, gdzie n = 1, 2, 3 $\dots$ 

In [12]:
function f(x::Float64)
    return sqrt(x^2.0 + 1.0) - 1.0
end

function g(x::Float64) 
    return x^2.0/(sqrt(x^2.0 + 1.0) + 1.0)
end
        
        
function compare(results::Int)
    
    for i = 1:results
        println("f(x): ", f((1/8)^i),"  g(x): ", g((1/8)^i))
    end
    
end
    

compare (generic function with 1 method)

In [15]:
compare(20)

f(x): 0.0077822185373186414  g(x): 0.0077822185373187065
f(x): 0.00012206286282867573  g(x): 0.00012206286282875901
f(x): 1.9073468138230965e-6  g(x): 1.907346813826566e-6
f(x): 2.9802321943606103e-8  g(x): 2.9802321943606116e-8
f(x): 4.656612873077393e-10  g(x): 4.6566128719931904e-10
f(x): 7.275957614183426e-12  g(x): 7.275957614156956e-12
f(x): 1.1368683772161603e-13  g(x): 1.1368683772160957e-13
f(x): 1.7763568394002505e-15  g(x): 1.7763568394002489e-15
f(x): 0.0  g(x): 2.7755575615628914e-17
f(x): 0.0  g(x): 4.336808689942018e-19
f(x): 0.0  g(x): 6.776263578034403e-21
f(x): 0.0  g(x): 1.0587911840678754e-22
f(x): 0.0  g(x): 1.6543612251060553e-24
f(x): 0.0  g(x): 2.5849394142282115e-26
f(x): 0.0  g(x): 4.0389678347315804e-28
f(x): 0.0  g(x): 6.310887241768095e-30
f(x): 0.0  g(x): 9.860761315262648e-32
f(x): 0.0  g(x): 1.5407439555097887e-33
f(x): 0.0  g(x): 2.407412430484045e-35
f(x): 0.0  g(x): 3.76158192263132e-37


Od pewnego momentu (już dla n = 9) funkcja f zaczyna zwracać 0.0. Podczas obliczania ($f(x) = \sqrt{x^2 + 1} - 1$) następuje redukcja cyfr znaczących, odejmowane są bliskie sobie liczby. Aby uniknąć błędów obliczeniowych, należy unikać odejmowania liczb znajdujących się blisko siebie, przykładowo zmieniając sposób w jaki opisujemy zadany problem.

# Zadanie 7

Policz przybliżoną wartość pochodnej $f(x) = sinx + cos3x$ za pomocą wzoru $f'(x_0) = \frac{f(x_0 + h) - f(x_0)}{h}$. Wyniki porównaj z dokładną wartością pochodnej.

W poniższych obliczeniach będę rozważał $h$ zgodnie ze wzorem: $h = 2^{-n}$ dla n = 0, 1, 2, $\dots$, 54. 

In [20]:
function approx_derivative(func, x0, h)
    return (func(x0 + h) - func(x0)) / h
end


function func(x)
    return sin(x) + cos(3x)
end

   
function compare_derivative(x0::Float64)
    
    derivative = Float64(cos(x0) - 3 * sin(3 * x0))
    best::Float64 = 1.0
    result::Float64 = 0.0
    for n = 0 : 54
        h = Float64((1/2)^n)
        approx = approx_derivative(func, x0, h)
        error = abs(derivative - approx)
        println("Dla h = ", h, ", wartość przybliżanej pochodnej wynosi: ", approx)
        println("Błąd takiego przybliżenia to: ", error)
        if best > error
            best = error
            result = approx
        end
    end
    
    println("")
    println("Dokładna wartość pochodnej wynosi: ", derivative)
    println("Najlepsze przybliżenie pochodnej wynosi: ", result, " z błędem ", best)
    
end
     

compare_derivative (generic function with 1 method)

In [19]:
compare_derivative(1.0)

Dla h = 1.0 0, wartość przybliżanej pochodnej wynosi: 2.0179892252685967
Błąd takiego przybliżenia to: 1.9010469435800585
Dla h = 0.5 1, wartość przybliżanej pochodnej wynosi: 1.8704413979316472
Błąd takiego przybliżenia to: 1.753499116243109
Dla h = 0.25 2, wartość przybliżanej pochodnej wynosi: 1.1077870952342974
Błąd takiego przybliżenia to: 0.9908448135457593
Dla h = 0.125 3, wartość przybliżanej pochodnej wynosi: 0.6232412792975817
Błąd takiego przybliżenia to: 0.5062989976090435
Dla h = 0.0625 4, wartość przybliżanej pochodnej wynosi: 0.3704000662035192
Błąd takiego przybliżenia to: 0.253457784514981
Dla h = 0.03125 5, wartość przybliżanej pochodnej wynosi: 0.24344307439754687
Błąd takiego przybliżenia to: 0.1265007927090087
Dla h = 0.015625 6, wartość przybliżanej pochodnej wynosi: 0.18009756330732785
Błąd takiego przybliżenia to: 0.0631552816187897
Dla h = 0.0078125 7, wartość przybliżanej pochodnej wynosi: 0.1484913953710958
Błąd takiego przybliżenia to: 0.03154911368255764
Dl

Zmniejszanie $h$ od pewnego momentu (dla n = 28) nie powoduje poprawy wyniku. Przyczyną jest suma pojawiająca się we wzorze $f(x_0 + h)$, która jest podatna na zjawisko pochłaniania (dla podanej arytmetyki zachodzi $1 + h = 1$, dla bardzo małych $h$). Powinniśmy zatem unikać dodawania liczb znacząco rózniących się rzędem. 